In [122]:
import os
import pandas as pd

BASE = "results"
GO_DIR = os.path.join(BASE, "enrichment_go")
KEGG_DIR = os.path.join(BASE, "enrichment_kegg")
MAP_DIR = os.path.join(BASE, "id_mapping")
os.makedirs(GO_DIR, exist_ok=True)
os.makedirs(KEGG_DIR, exist_ok=True)
os.makedirs(MAP_DIR, exist_ok=True)

degs_all = pd.read_csv("results/degs_all.csv")
degs_up  = pd.read_csv("results/degs_up.csv")
dge      = pd.read_csv("results/dge_results.csv")

def uniq_list(x):
    x = pd.Series(x).dropna().astype(str).str.strip()
    x = x[x != ""]
    return list(dict.fromkeys(x.tolist()))

all_probes = uniq_list(degs_all["gene_id"])
up_probes  = uniq_list(degs_up["gene_id"])
down_probes = uniq_list(dge[(dge["log2FoldChange"] < 0) & (dge["padj"] < 0.05)]["gene_id"])

print("Probe sizes:", len(all_probes), len(up_probes), len(down_probes))
print("Preview probes:", all_probes[:15])

Probe sizes: 60 46 1242
Preview probes: ['7961102', '8151592', '8095694', '8156706', '8063478', '8131067', '8092552', '8173135', '8038877', '8062312', '7953901', '8161373', '8161554', '8021081', '8071044']


##  Correct probe→symbol mapping

In [125]:
import mygene
mg = mygene.MyGeneInfo()

def map_probes_to_symbols(probes, species):
    """
    Robust mapping:
    - Always creates probe_id from df.index safely
    - Tries reporter first, then broader scopes
    Returns:
      mapped_df with columns [probe_id, symbol, entrezgene, name, taxid]
      symbols list (unique)
    """
    scopes_try = [
        ["reporter"],                          
        ["reporter", "symbol", "alias"],
        ["reporter", "accession", "refseq"],
        ["symbol", "alias", "refseq", "accession"],  
    ]

    for scopes in scopes_try:
        df = mg.querymany(
            probes,
            scopes=scopes,
            fields="symbol,entrezgene,name,taxid",
            species=species,
            as_dataframe=True,
            returnall=False,
            verbose=False,
        )

        if not isinstance(df, pd.DataFrame) or df.shape[0] == 0:
            continue

        # df index contains the original query values; make it an explicit column
        df2 = df.copy()
        df2["probe_id"] = df2.index.astype(str)

        # Keep only rows with symbol
        if "symbol" not in df2.columns:
            continue

        mapped = df2.dropna(subset=["symbol"]).copy()
        if mapped.shape[0] == 0:
            continue

        mapped["symbol"] = mapped["symbol"].astype(str).str.strip()
        mapped = mapped[["probe_id", "symbol", "entrezgene", "name", "taxid"]].drop_duplicates()

        symbols = list(dict.fromkeys(mapped["symbol"].tolist()))
        return mapped, symbols

    return pd.DataFrame(columns=["probe_id","symbol","entrezgene","name","taxid"]), []

mapped_h, symbols_h = map_probes_to_symbols(all_probes, species="human")
mapped_m, symbols_m = map_probes_to_symbols(all_probes, species="mouse")

print("Human mapped probes:", mapped_h.shape[0], "| unique symbols:", len(symbols_h))
print("Mouse mapped probes:", mapped_m.shape[0], "| unique symbols:", len(symbols_m))

# choose best
if mapped_h.shape[0] >= mapped_m.shape[0]:
    SPECIES = "human"
    ORGANISM = "Human"
    probe2sym_df = mapped_h
else:
    SPECIES = "mouse"
    ORGANISM = "Mouse"
    probe2sym_df = mapped_m

print("Chosen:", SPECIES, ORGANISM)

probe2sym_df.to_csv(os.path.join(MAP_DIR, f"probe_to_symbol_{SPECIES}.csv"), index=False)
probe_to_symbol = dict(zip(probe2sym_df["probe_id"].astype(str), probe2sym_df["symbol"].astype(str)))

Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed


Human mapped probes: 86 | unique symbols: 55
Mouse mapped probes: 0 | unique symbols: 0
Chosen: human Human


## Convert ALL/UP/DOWN probes → symbols

In [128]:
def probes_to_symbols(probes, mapping):
    out = []
    for p in probes:
        s = mapping.get(str(p))
        if s is not None and str(s).strip() != "":
            out.append(str(s).strip())
    return list(dict.fromkeys(out))

all_genes  = probes_to_symbols(all_probes, probe_to_symbol)
up_genes   = probes_to_symbols(up_probes, probe_to_symbol)
down_genes = probes_to_symbols(down_probes, probe_to_symbol)

print("Mapped symbol sizes:", len(all_genes), len(up_genes), len(down_genes))
print("Preview ALL symbols:", all_genes[:25])

pd.Series(all_genes).to_csv(os.path.join(BASE, "enrich_all_symbols.csv"), index=False, header=["symbol"])
pd.Series(up_genes).to_csv(os.path.join(BASE, "enrich_up_symbols.csv"), index=False, header=["symbol"])
pd.Series(down_genes).to_csv(os.path.join(BASE, "enrich_down_symbols.csv"), index=False, header=["symbol"])

if len(all_genes) < 5:
    raise ValueError(
        "Too few gene symbols mapped. This suggests MyGene cannot map these probe IDs.\n"
        "If this happens, the correct solution is to use the GPL6244 annotation file directly."
    )

Mapped symbol sizes: 42 33 9
Preview ALL symbols: ['CLEC1B', 'CA1', 'PF4V1', 'TMOD1', 'MIMS2', 'GPR146', 'IGF2BP2', 'ALAS2', 'SIGLEC5', 'MYL9', 'CLEC12A', 'LINC00268', 'SLC14A1', 'LOC100233156', 'MYL4', 'HEMGN', 'GLRX5', 'TREML1', 'PRKAR2B', 'RPS26P11', 'CCDC144BP', 'DEFA1B', 'DEFA1', 'PPBP', 'RHD']


## Enrichr ID test (must be > 0 before running full GO/KEGG)

In [131]:
import gseapy as gp

test = gp.enrichr(
    gene_list=all_genes[:50],
    gene_sets=["GO_Biological_Process_2023"],
    organism=ORGANISM,
    outdir=os.path.join(GO_DIR, f"id_test_{ORGANISM.lower()}"),
    cutoff=1.0,
    no_plot=True
)
print("ID test rows:", test.results.shape[0])
test.results.head()

ID test rows: 227


,Gene_set,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes
0,GO_Biological_Process_2023,Antimicrobial Humoral Immune Response Mediated...,4/65,0.000010,0.002352,0,0,34.334771,394.070221,DEFA4;PPBP;DEFA1;PF4V1
1,GO_Biological_Process_2023,Antimicrobial Humoral Response (GO:0019730),4/100,0.000057,0.006460,0,0,21.778509,212.863194,DEFA4;PPBP;DEFA1;PF4V1
2,GO_Biological_Process_2023,Cellular Response To Molecule Of Bacterial Ori...,4/117,0.000105,0.007451,0,0,18.486260,169.387243,DEFA4;PPBP;DEFA1;PF4V1
3,GO_Biological_Process_2023,Cellular Response To Lipopolysaccharide (GO:00...,4/124,0.000131,0.007451,0,0,17.401754,155.538776,DEFA4;PPBP;DEFA1;PF4V1
4,GO_Biological_Process_2023,Response To Lipopolysaccharide (GO:0032496),4/159,0.000340,0.010940,0,0,13.448557,107.398216,DEFA4;PPBP;DEFA1;PF4V1


## GO enrichment

In [134]:
GO_SETS = [
    "GO_Biological_Process_2023",
    "GO_Molecular_Function_2023",
    "GO_Cellular_Component_2023",
]

def run_go(glist, name):
    outdir = os.path.join(GO_DIR, name)
    os.makedirs(outdir, exist_ok=True)

    enr = gp.enrichr(
        gene_list=glist,
        gene_sets=GO_SETS,
        organism=ORGANISM,
        outdir=outdir,
        cutoff=1.0,
        no_plot=True
    )

    res = enr.results.copy()
    res.to_csv(os.path.join(outdir, f"{name}_go_ALLTERMS.csv"), index=False)

    sig = res[res["Adjusted P-value"] < 0.05].copy()
    sig.to_csv(os.path.join(outdir, f"{name}_go_SIG_FDR005.csv"), index=False)

    print(f"[GO:{name}] total={res.shape[0]} | sig={sig.shape[0]}")
    return res, sig

go_all_res, go_all_sig   = run_go(all_genes, "all")
go_up_res, go_up_sig     = run_go(up_genes, "up")
go_down_res, go_down_sig = run_go(down_genes, "down")

go_all_res.head()

[GO:all] total=349 | sig=31
[GO:up] total=320 | sig=49
[GO:down] total=50 | sig=27


,Gene_set,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes
0,GO_Biological_Process_2023,Antimicrobial Humoral Immune Response Mediated...,4/65,0.000010,0.002352,0,0,34.334771,394.070221,DEFA4;PPBP;DEFA1;PF4V1
1,GO_Biological_Process_2023,Antimicrobial Humoral Response (GO:0019730),4/100,0.000057,0.006460,0,0,21.778509,212.863194,DEFA4;PPBP;DEFA1;PF4V1
2,GO_Biological_Process_2023,Cellular Response To Molecule Of Bacterial Ori...,4/117,0.000105,0.007451,0,0,18.486260,169.387243,DEFA4;PPBP;DEFA1;PF4V1
3,GO_Biological_Process_2023,Cellular Response To Lipopolysaccharide (GO:00...,4/124,0.000131,0.007451,0,0,17.401754,155.538776,DEFA4;PPBP;DEFA1;PF4V1
4,GO_Biological_Process_2023,Response To Lipopolysaccharide (GO:0032496),4/159,0.000340,0.010940,0,0,13.448557,107.398216,DEFA4;PPBP;DEFA1;PF4V1


In [139]:
import os
import pandas as pd
import gseapy as gp

GO_PLOT_DIR = os.path.join(GO_DIR, "plots")
os.makedirs(GO_PLOT_DIR, exist_ok=True)

def go_barplot(sig_df, name, top_n=15):
    """Barplot top GO terms (safe: skips if empty)."""
    if sig_df is None or sig_df.shape[0] == 0:
        print(f"[PLOT:GO:{name}] No significant terms to plot.")
        return None

    df = sig_df.copy()
    df["Adjusted P-value"] = pd.to_numeric(df["Adjusted P-value"], errors="coerce")
    df = df.dropna(subset=["Adjusted P-value"]).sort_values("Adjusted P-value").head(top_n)

    outpath = os.path.join(GO_PLOT_DIR, f"go_{name}_top{top_n}_bar.png")
    gp.barplot(
        df,
        title=f"GO Enrichment ({name}) — Top {top_n}",
        ofname=outpath
    )
    print(f"[PLOT:GO:{name}] Saved:", outpath)
    return outpath

def go_dotplot(sig_df, name, top_n=15):
    """Dotplot top GO terms (safe: skips if empty)."""
    if sig_df is None or sig_df.shape[0] == 0:
        print(f"[PLOT:GO:{name}] No significant terms to plot.")
        return None

    df = sig_df.copy()
    df["Adjusted P-value"] = pd.to_numeric(df["Adjusted P-value"], errors="coerce")
    df = df.dropna(subset=["Adjusted P-value"]).sort_values("Adjusted P-value").head(top_n)

    outpath = os.path.join(GO_PLOT_DIR, f"go_{name}_top{top_n}_dot.png")
    gp.dotplot(
        df,
        column="Adjusted P-value",
        title=f"GO Enrichment ({name}) — Top {top_n}",
        ofname=outpath
    )
    print(f"[PLOT:GO:{name}] Saved:", outpath)
    return outpath

go_barplot(go_all_sig,  "all",  top_n=15)
go_barplot(go_up_sig,   "up",   top_n=15)
go_barplot(go_down_sig, "down", top_n=15)

go_dotplot(go_all_sig,  "all",  top_n=15)
go_dotplot(go_up_sig,   "up",   top_n=15)
go_dotplot(go_down_sig, "down", top_n=15)

[PLOT:GO:all] Saved: results/enrichment_go/plots/go_all_top15_bar.png
[PLOT:GO:up] Saved: results/enrichment_go/plots/go_up_top15_bar.png
[PLOT:GO:down] Saved: results/enrichment_go/plots/go_down_top15_bar.png
[PLOT:GO:all] Saved: results/enrichment_go/plots/go_all_top15_dot.png
[PLOT:GO:up] Saved: results/enrichment_go/plots/go_up_top15_dot.png
[PLOT:GO:down] Saved: results/enrichment_go/plots/go_down_top15_dot.png


'results/enrichment_go/plots/go_down_top15_dot.png'

## KEGG enrichment

In [137]:
KEGG_LIBRARY = "KEGG_2021_Human" if ORGANISM == "Human" else "KEGG_2021_Mouse"

def run_kegg(glist, name):
    outdir = os.path.join(KEGG_DIR, name)
    os.makedirs(outdir, exist_ok=True)

    enr = gp.enrichr(
        gene_list=glist,
        gene_sets=[KEGG_LIBRARY],
        organism=ORGANISM,
        outdir=outdir,
        cutoff=1.0,
        no_plot=True
    )

    res = enr.results.copy()
    res.to_csv(os.path.join(outdir, f"{name}_kegg_ALLTERMS.csv"), index=False)

    sig = res[res["Adjusted P-value"] < 0.05].copy()
    sig.to_csv(os.path.join(outdir, f"{name}_kegg_SIG_FDR005.csv"), index=False)

    print(f"[KEGG:{name}] total={res.shape[0]} | sig={sig.shape[0]}")
    return res, sig

kegg_all_res, kegg_all_sig   = run_kegg(all_genes, "all")
kegg_up_res, kegg_up_sig     = run_kegg(up_genes, "up")
kegg_down_res, kegg_down_sig = run_kegg(down_genes, "down")

kegg_all_res.head()

[KEGG:all] total=66 | sig=16
[KEGG:up] total=64 | sig=19
[KEGG:down] total=3 | sig=3


,Gene_set,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes
0,KEGG_2021_Human,Staphylococcus aureus infection,5/95,0.000002,0.000106,0,0,29.831832,397.947934,HLA-DRB5;DEFA4;DEFA1;DEFA1B;HLA-DRB1
1,KEGG_2021_Human,Hematopoietic cell lineage,3/99,0.001174,0.034708,0,0,15.915064,107.383782,HLA-DRB5;GYPA;HLA-DRB1
2,KEGG_2021_Human,Asthma,2/31,0.001926,0.034708,0,0,34.360345,214.830882,HLA-DRB5;HLA-DRB1
3,KEGG_2021_Human,Allograft rejection,2/38,0.002885,0.034708,0,0,27.669444,161.818408,HLA-DRB5;HLA-DRB1
4,KEGG_2021_Human,"Glycine, serine and threonine metabolism",2/40,0.003192,0.034708,0,0,26.210526,150.631615,ALAS2;PSPH


In [141]:
import os
import pandas as pd
import gseapy as gp

PLOT_DIR = os.path.join(KEGG_DIR, "plots")
os.makedirs(PLOT_DIR, exist_ok=True)

def kegg_barplot(sig_df, name, top_n=15):
    """Barplot top KEGG terms (safe: skips if empty)."""
    if sig_df is None or sig_df.shape[0] == 0:
        print(f"[PLOT:KEGG:{name}] No significant terms to plot.")
        return None

    df = sig_df.copy()
    df["Adjusted P-value"] = pd.to_numeric(df["Adjusted P-value"], errors="coerce")
    df = df.dropna(subset=["Adjusted P-value"]).sort_values("Adjusted P-value").head(top_n)

    outpath = os.path.join(PLOT_DIR, f"kegg_{name}_top{top_n}_bar.png")
    gp.barplot(
        df,
        title=f"KEGG Enrichment ({name}) — Top {top_n}",
        ofname=outpath
    )
    print(f"[PLOT:KEGG:{name}] Saved:", outpath)
    return outpath

def kegg_dotplot(sig_df, name, top_n=15):
    """Dotplot top KEGG terms (safe: skips if empty)."""
    if sig_df is None or sig_df.shape[0] == 0:
        print(f"[PLOT:KEGG:{name}] No significant terms to plot.")
        return None

    df = sig_df.copy()
    df["Adjusted P-value"] = pd.to_numeric(df["Adjusted P-value"], errors="coerce")
    df = df.dropna(subset=["Adjusted P-value"]).sort_values("Adjusted P-value").head(top_n)

    outpath = os.path.join(PLOT_DIR, f"kegg_{name}_top{top_n}_dot.png")
    gp.dotplot(
        df,
        column="Adjusted P-value",
        title=f"KEGG Enrichment ({name}) — Top {top_n}",
        ofname=outpath
    )
    print(f"[PLOT:KEGG:{name}] Saved:", outpath)
    return outpath

kegg_barplot(kegg_all_sig,  "all",  top_n=15)
kegg_barplot(kegg_up_sig,   "up",   top_n=15)
kegg_barplot(kegg_down_sig, "down", top_n=15)

kegg_dotplot(kegg_all_sig,  "all",  top_n=15)
kegg_dotplot(kegg_up_sig,   "up",   top_n=15)
kegg_dotplot(kegg_down_sig, "down", top_n=15)

[PLOT:KEGG:all] Saved: results/enrichment_kegg/plots/kegg_all_top15_bar.png
[PLOT:KEGG:up] Saved: results/enrichment_kegg/plots/kegg_up_top15_bar.png
[PLOT:KEGG:down] Saved: results/enrichment_kegg/plots/kegg_down_top15_bar.png
[PLOT:KEGG:all] Saved: results/enrichment_kegg/plots/kegg_all_top15_dot.png
[PLOT:KEGG:up] Saved: results/enrichment_kegg/plots/kegg_up_top15_dot.png
[PLOT:KEGG:down] Saved: results/enrichment_kegg/plots/kegg_down_top15_dot.png


'results/enrichment_kegg/plots/kegg_down_top15_dot.png'